In [1]:
import os
os.environ["TRITON_INTERPRET"] = "1"

import torch
N, D = 8, 4
aa = torch.randn((N, D), device="cuda")

import triton
import triton.language as tl

from model_triton import gelu_k, T_DROPOUT_RATE, dropout_k

import triton.language as core
@triton.jit
def _indicator(n_dims: core.constexpr, idx: core.constexpr, pos: core.constexpr):
    core.static_assert(idx < n_dims)
    core.static_assert((pos == 0) or (pos == 1))
    y = core.arange(0, 2)
    if pos == 0:
        y = 1 - y
    for n in core.static_range(0, n_dims):
        if n != n_dims - 1 - idx:
            y = core.expand_dims(y, n)
    return y

@triton.jit
def _take_slice(x, n_dims: core.constexpr, idx: core.constexpr, pos: core.constexpr, keep_dim: core.constexpr = True):
    y = tl.sum(x * _indicator(n_dims, idx, pos), n_dims - 1 - idx)
    if keep_dim:
        y = core.expand_dims(y, n_dims - 1 - idx)

    return y

from triton_extensions import _take_slice_, _put_slice_
# @triton.jit
# def _indicator_(n_dims: core.constexpr, idx: core.constexpr, pos: core.constexpr, pos_dim: core.constexpr):
#     core.static_assert(idx < n_dims)
#     core.static_assert((pos>0) and (pos <= pos_dim))
#     y = core.arange(1, pos_dim+1)
#     y = tl.where(y==pos, 1, 0)
    
#     for n in core.static_range(0, n_dims):
#         if n != n_dims - 1 - idx:
#             y = core.expand_dims(y, n)
#     return y

# @triton.jit
# def _take_slice_(x, n_dims: core.constexpr, idx: core.constexpr, pos: core.constexpr, pos_dim:core.constexpr, keep_dim: core.constexpr = True):
#     ind = _indicator_(n_dims, idx, pos+1, pos_dim)
#     y = tl.sum(x * ind, n_dims - 1 - idx)
#     if keep_dim:
#         y = core.expand_dims(y, n_dims - 1 - idx)

#     return y

# @triton.jit
# def _put_slice_(x, n_dims: core.constexpr, idx: core.constexpr, pos: core.constexpr, pos_dim:core.constexpr, input_slice):
#     ind = _indicator_(n_dims, idx, pos+1, pos_dim)
#     print(f'x', x.shape)
#     print(f'input_slice', input_slice.shape)
#     print(f'ind', ind.shape, ind)
#     y = tl.where(ind, input_slice, x)
#     return y

@triton.jit
def my_kernel(input_ptr, input_stride0, input_stride1, output_ptr, 
              N:tl.constexpr, D:tl.constexpr):
    n_offsets = tl.arange(0, N)            
    d_offsets = tl.arange(0, D)
    input_blck_ptr = input_ptr + n_offsets[:,None] * input_stride0 + d_offsets[None,:] * input_stride1
    input_blck_mask = (n_offsets[:,None] < N) & (d_offsets[None, :] < D)
    input_blck = tl.load(input_blck_ptr, mask=input_blck_mask, other=0.0)
    #print(f'input_blck', input_blck)
    
    #output = _take_slice(input_blck, 2, 0, 0, keep_dim=False)
    output = _take_slice_(input_blck, 2, 0, 1, 4, keep_dim=False) # take column
    print(f'output', output.shape)
    #output = _take_slice_(input_blck, 2, 1, 1, 8, keep_dim=False) # take row 
    
    # copying column slice
    #slice1 = _take_slice_(input_blck, 2, 0, 1, 4, keep_dim=True) # take column
    #input_blck = _put_slice_(input_blck, 2, 0, 2, 4, slice1)    
    #print(f'input_blck', input_blck)
    
    # copying row slice
    slice1 = _take_slice_(input_blck, 2, 1, 1, 8, keep_dim=True)
    input_blck = _put_slice_(input_blck, 2, 1, 2, 8, slice1)    
    #print(f'input_blck', input_blck)
    
    #print('output', output)
    tl.store(output_ptr + n_offsets, output, mask=n_offsets<N)    
    
    
print('aa', aa)
output = torch.zeros((N), device="cuda")
my_kernel[(1,)](aa, aa.stride(0), aa.stride(1), output, N, D)
print('output', output)

aa tensor([[-0.0208,  1.6458, -0.0943, -1.7180],
        [ 0.3743, -0.5518,  0.6232,  0.2076],
        [ 0.5863,  0.2572, -0.4931, -0.6361],
        [ 1.1984,  0.5690,  0.0333,  0.4207],
        [ 1.4335,  1.2339,  0.6477, -1.0560],
        [ 2.6699, -3.0328, -0.9142,  0.4571],
        [ 0.5359,  1.7751, -1.0170, -1.0332],
        [-1.4559,  1.2538, -1.3986, -0.2163]], device='cuda:0')
output [constexpr[8]]
output tensor([ 1.6458, -0.5518,  0.2572,  0.5690,  1.2339, -3.0328,  1.7751,  1.2538],
       device='cuda:0')
